In [7]:
import numpy as np
import json
import time
import os

from pymongo import MongoClient
import pygad

import matplotlib.pyplot as plt



In [8]:
def get_pokemon_individually():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # Individual pokemon's collection
    collec_indiv_pokemon = db['pokemons']
    pokemons = list(collec_indiv_pokemon.find())
    
    return pokemons

def get_pokemon_teams():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # pokemon's teams collection
    collec_poke_teams = db['pokemon_teams']
    collec_poke_teams = list(collec_poke_teams.find())
    
    for poke_team in collec_poke_teams:
        del poke_team['_id']    
    return collec_poke_teams

def best_against(pokeA, pokeB):
    
    best = 0
    pokemons_max_types = ['primaryType', 'secondaryType']
    
    for poke_typeA in pokemons_max_types:
        if poke_typeA in pokeA:
            for poke_typeB in pokemons_max_types:
                if poke_typeB in pokeB:
                    typeA_against_typeB = f'{poke_typeA}_against_{pokeB[poke_typeB]}'
                    if typeA_against_typeB in pokeA and pokeA[typeA_against_typeB] > best:

                        best = pokeA[typeA_against_typeB]
    return best


def battle(poke1, poke2):

    against1 = best_against(poke1, poke2)
    against2 = best_against(poke2, poke1)
    
    return (poke1['CP'] * against1) - (poke2['CP'] * against2)

# Get all pokemons in a list
pokemons = get_pokemon_individually()




def create_log_dirs(team_idx, ga_instance_run, run_number):

    # Diretório principal de resultados
    target_dir = os.path.join(os.getcwd(), 'Resultados')

    # Garantir que o diretório "Resultados" exista
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Diretório específico do time
    folder_name = os.path.join(target_dir, f'RivalTeam_{team_idx}')
    
    # Garantir que o diretório do time exista
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)


    

    # Caminho onde o resultado da execução será salvo
    folder_target_ga_instance = os.path.join(folder_name, f'Executution_{run_number}')

    # Salvando a instância do GA no diretório
    ga_instance_run.save(folder_target_ga_instance)

    plt.figure()
    plt.plot(ga_instance_run.best_solutions_fitness)

    # Caminho para salvar o gráfico
    folder_target_fitness_plot = os.path.join(folder_name, f'Execution_{run_number}.jpeg')

    # Salvando o gráfico no diretório correto
    plt.savefig(folder_target_fitness_plot, dpi=400)



    # Fechar a figura para evitar problemas com novos plots
    plt.close()



In [11]:
def fitness_func(ga_instance, solution, solution_idx):
    # Starts the fit value with 0
    fit = 0

    # Getting the rival team from the global variables
    global rival_team

    #print(solution)
    solution_team = [pokemons[solution_pokemon] for solution_pokemon in solution]
    for home_pokemon in solution_team:
        for rival_pokemon in rival_team:
            fit += battle(home_pokemon, rival_pokemon)

    return fit

parent_selection_type = 'rws'
crossover_type = "single_point"
mutation_type="random"

def run_genetic_algorithm():
    poke_rival_teams = get_pokemon_teams()[:20]

    for rival_team_idx, rv_team_db in enumerate(poke_rival_teams):

        # Creating the global rival_team variable
        globals()['rival_team'] =  rv_team_db[f'Team {rival_team_idx}']

        for run_number in range(5):
            ga_instance = pygad.GA(num_generations = 30,
                                num_parents_mating =  400,
                                fitness_func = fitness_func,
                                gene_type = int,
                                init_range_low = 0,
                                init_range_high = len(pokemons),
                                gene_space = np.arange(len(pokemons)),
                                sol_per_pop = 1000,
                                num_genes = 3,
                                parent_selection_type = parent_selection_type,
                                crossover_type=crossover_type,
                                crossover_probability = 0.8,
                                mutation_probability = 0.2,
                                mutation_by_replacement = True,
                                mutation_type = mutation_type,
                                random_mutation_min_val = 0,
                                random_mutation_max_val = len(pokemons),
                                save_solutions = True,
                                keep_parents = 1)
            

            ga_instance.run()
            

            create_log_dirs(rival_team_idx, ga_instance, run_number)


# Rodar o algoritmo
run_genetic_algorithm()





c:\Users\gabri\anaconda3\envs\tcc\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\gabri\anaconda3\envs\tcc\Lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in populat